In [59]:
import math
import numpy as np

import numpy.linalg as npl
import matplotlib.pyplot as plt
#import matplotlib.animation as anim

import cv2

#from dataclasses import dataclass, field
#!pip install keyboard
import dataclasses as dat
import numpy.linalg as npl
#import keyboard as key
from PIL import Image, ImageDraw,ImageFont

In [60]:
@dat.dataclass
class point():
    glav: bool = 0
    x_prev: float = 0
    y_prev: float = 0
    x: float = 0
    y: float = 0
    vx: float = 0
    vy: float = 0
    alpha1: float=0
    alpha2: float=0
    alpha3: float=0
    alpha4: float=0
    alpha5: float=0
    catched: bool=0

@dat.dataclass
class enemy():
  exist: bool=1
  x_prev: float=0
  y_prev: float=0
  x: float=0
  y: float=0
  vx: float=0
  vy: float=0
  alpha: float=0

@dat.dataclass
class segment():
    ax: float = 0
    ay: float = 0
    bx: float = 0
    by: float = 0

@dat.dataclass
class circle():
    cx: float = 0
    cy: float = 0
    radius: float = 0

def circle_ro(cr,m):
    l=math.sqrt(abs((m.x-cr.cx)**2+(m.y-cr.cy)**2))
    ans=abs(l-cr.radius)
    return(ans)
 
def segm_ro(ab,m) : 
    A=[ab.ax,ab.ay]
    B=[ab.bx,ab.by]
    E=[m.x,m.y]
    # vector AB 
    AB = [None, None] 
    AB[0] = B[0] - A[0]
    AB[1] = B[1] - A[1]
    # vector BP 
    BE = [None, None]
    BE[0] = E[0] - B[0]
    BE[1] = E[1] - B[1]
    # vector AP 
    AE = [None, None]
    AE[0] = E[0] - A[0]
    AE[1] = E[1] - A[1]
 
    # Variables to store dot product 
    # Calculating the dot product 
    AB_BE = AB[0] * BE[0] + AB[1] * BE[1]; 
    AB_AE = AB[0] * AE[0] + AB[1] * AE[1]; 
 
    # Minimum distance from point E to the line segment 
    ans = 0; 
    # Case 1 
    if (AB_BE > 0) :
        # Finding the magnitude 
        y = E[1] - B[1]; 
        x = E[0] - B[0]; 
        ans = math.sqrt(x * x + y * y); 
    # Case 2 
    elif (AB_AE < 0) :
        y = E[1] - A[1]; 
        x = E[0] - A[0]; 
        ans = math.sqrt(x * x + y * y); 
 
    # Case 3 
    else:
        # Finding the perpendicular distance 
        x1 = AB[0]; 
        y1 = AB[1]; 
        x2 = AE[0]; 
        y2 = AE[1]; 
        mod = math.sqrt(x1 * x1 + y1 * y1); 
        ans = abs(x1 * y2 - y1 * x2) / mod; 
    return ans

In [61]:
eps=10

#p0=point(198,250,50,-50)
#p1=point(252,150,50,-50)

obst0 = segment(50,60,200,110)
obst1 = segment(190,170,190,220)
obst2 = circle(62,210,20)
obst3 = circle(110,375,50)
obst4 = segment(300,260,440,400)
obst5 = segment(490,170,260,230)
obst6 = circle(352,400,40)
#obst7 = segment(270,410,270,450)

segs = [obst0,obst1,obst4,obst5]
cirs = [obst2,obst3,obst6]

In [62]:
tfin=1000
p=25
w=500
r=5
n=1.5
dt=0.75
loss=0
p_new=p/n
col4=(204,0,0)
col3=(255,146,24)
col2=(0,0,0)
col1=(255,255,255)
images=[]
im=Image.new('RGB',(w,w),col1)
images.append(im)


b0 = point(1,0,0,w/2+0,w/2+0,1,2,0.5,1.0,0.6,0.4,0.0,0)
b1 = point(0,0,0,w/2+3*p_new,w/2+2*p_new,0,1,0.5,0.0,0.1,0.7,1.0,0)
b2 = point(0,0,0,w/2-2*p_new,w/2+3*p_new,1/np.sqrt(2),1/np.sqrt(2),0.5,0.0,0.1,0.7,1.0,0)
b=[b0,b1,b2]
en=enemy(0,0,0,w-2*p_new,w-2*p_new,3/np.sqrt(2),3/np.sqrt(2),3.0)
t=0
while((t<tfin)and(not(loss))):
  k=0
  #Поворот вектора скорости при отражении от кружочка
  for k in range(len(b)):
    for obj in cirs:
        #print(b[k])
        #print(obj)
        #print('dist=',circle_ro(obj,b[k]))
        if circle_ro(obj,b[k])<eps:
            theory_point=point(b[k].glav,b[k].x,b[k].y,b[k].x+b[k].vx*dt,b[k].y+b[k].vy*dt,b[k].vx,b[k].vy,b[k].alpha1,b[k].alpha2,b[k].alpha3,b[k].alpha4,b[k].alpha5,b[k].catched)
            if circle_ro(obj,theory_point)<circle_ro(obj,b[k]):
              print('circle turn')
              print(b[k])
              u = np.array([b[k].vx, b[k].vy, 0])   # velocity
              rad = np.array([obj.cx-b[k].x, obj.cy-b[k].y, 0])   # radius
              tan = np.cross(rad, [0, 0, 1]) # tangent
          
              r_norm = np.sqrt(sum(rad**2))     
              proj_of_u_on_r = (np.dot(u, rad)/r_norm**2)*rad
              t_norm = np.sqrt(sum(tan**2))     
              proj_of_u_on_t = (np.dot(u, tan)/t_norm**2)*tan
              
              #print("Projection of velocity on radius is: ", proj_of_u_on_r)
              #print("Projection of velocity on tangent is: ", proj_of_u_on_t)
              u_new =  np.array([-proj_of_u_on_r[0]+proj_of_u_on_t[0], -proj_of_u_on_r[1]+proj_of_u_on_t[1], 0])
              #print("Old velocity: [", u[0],u[1],"]")
              #print("New velocity: [", u_new[0],u_new[1],"]")
              b[k].vx=u_new[0]
              b[k].vy=u_new[1]
              print(b[k])
              print('\n')
  k=0

  #Поворот вектора скорости при отражении от отрезка
  for k in range(len(b)):
      for obj in segs:
          #print(b[k])
          #print(obj)
          #print('dist=',segm_ro(obj,b[k]))
          if segm_ro(obj,b[k])<eps:
              theory_point=point(b[k].glav,b[k].x,b[k].y,b[k].x+b[k].vx*dt,b[k].y+b[k].vy*dt,b[k].vx,b[k].vy,b[k].alpha1,b[k].alpha2,b[k].alpha3,b[k].alpha4,b[k].alpha5,b[k].catched)
              if segm_ro(obj,theory_point)<segm_ro(obj,b[k]):
                print('segm turn')
                print(b[k])
                u = np.array([b[k].vx, b[k].vy, 0])   # velocity
                rad = np.array([obj.bx-obj.ax, obj.by-obj.ay, 0])   # segment
                n = np.cross(rad, [0, 0, 1]) # normal vector
            
                r_norm = np.sqrt(sum(rad**2))     
                proj_of_u_on_r = (np.dot(u, rad)/r_norm**2)*rad
                n_norm = np.sqrt(sum(n**2))     
                proj_of_u_on_n = (np.dot(u, n)/n_norm**2)*n
                
                print("Projection of velocity on segment is: ", proj_of_u_on_r)
                print("Projection of velocity on normal vector is: ", proj_of_u_on_n)
                u_new =  np.array([proj_of_u_on_r[0]-proj_of_u_on_n[0], proj_of_u_on_r[1]-proj_of_u_on_n[1], 0])
                #print("Old velocity: [", u[0],u[1],"]")
                #print("New velocity: [", u_new[0],u_new[1],"]")
                b[k].vx=u_new[0]
                b[k].vy=u_new[1]
                print(b[k])
                print('\n')


              
  t=t+1*dt
  xc=np.array([b[0].x,b[0].y])
  vc=np.array([b[0].vx,b[0].vy])
  xen=np.array([en.x,en.y])
  target=0
  im=Image.new('RGB',(w,w),col1)
  draw=ImageDraw.Draw(im)
  draw.rectangle((0,0,w-1,w-1),fill=col1,outline=col2)

  for obj in cirs:
    draw.ellipse((obj.cx-obj.radius,obj.cy-obj.radius,obj.cx+obj.radius,obj.cy+obj.radius),fill=(154, 205, 50))

  for obj in segs:
    draw.line((obj.ax,obj.ay,obj.bx,obj.by),fill=(154, 205, 50), width=3)

  #for obj in pnts:
  #  draw.ellipse((obj.x-0.5*eps,obj.y-0.5*eps,obj.x+0.5*eps,obj.y+0.5*eps),fill=(0, 0, 0))
  




  if(((xc+b[0].alpha1*vc*dt)[0]>w-1-p_new) or ((xc+b[0].alpha1*vc*dt)[0]<p_new)):
    vc[0]=-vc[0]
  if(((xc+b[0].alpha1*vc*dt)[1]>w-1-p_new) or ((xc+b[0].alpha1*vc*dt)[1]<p_new)):
    vc[1]=-vc[1]
  b[0].x_prev=b[0].x
  b[0].y_prev=b[0].y
  prev=np.array([b[0].x_prev,b[0].y_prev])
  b[0].x=(xc+b[0].alpha1*vc*dt)[0]
  b[0].y=(xc+b[0].alpha1*vc*dt)[1]
  b[0].vx=(b[0].alpha2*vc+b[0].alpha3*(xc-prev)*(npl.norm(xc-prev,ord=2)>p)+b[0].alpha4*(-xc+prev)*(npl.norm(xc-prev,ord=2)<p))[0]
  b[0].vy=(b[0].alpha2*vc+b[0].alpha3*(xc-prev)*(npl.norm(xc-prev,ord=2)>p)+b[0].alpha4*(-xc+prev)*(npl.norm(xc-prev,ord=2)<p))[1]
  b[0].glav=1
  draw.ellipse((xc[0]-r,xc[1]-r,xc[0]+r,xc[1]+r),fill=col3)
  ren=npl.norm(xen-xc,ord=2)

  for k in range(1,len(b)):
    b[k].x_prev=b[k].x
    b[k].y_prev=b[k].y
    xk=np.array([b[k].x,b[k].y])
    prev=np.array([b[k].x_prev,b[k].y_prev])
    b[k].x=b[k].x_prev+b[k].alpha1*b[k].vx*dt
    b[k].y=b[k].y_prev+b[k].alpha1*b[k].vy*dt
    b[k].vx=b[k].alpha2*b[k].vx+b[k].alpha3*(xc-prev)[0]*(npl.norm(xc-prev,ord=2)>p)+b[k].alpha4*(-xc+prev)[0]*(npl.norm(xc-prev,ord=2)<p)
    b[k].vy=b[k].alpha2*b[k].vy+b[k].alpha3*(xc-prev)[1]*(npl.norm(xc-prev,ord=2)>p)+b[k].alpha4*(-xc+prev)[1]*(npl.norm(xc-prev,ord=2)<p)
    if(ren<npl.norm(xen-xk,ord=2)):
      ren=npl.norm(xen-xk,ord=2)
      target=k
    for j in range(0,k):
      #if(not(i==j)):
      xk=np.array([b[k].x,b[k].y])
      vk=np.array([b[k].vx,b[k].vy])
      xj=np.array([b[j].x,b[j].y])
      vj=np.array([b[j].vx,b[j].vy])
      if((npl.norm((xk+b[k].alpha1*vk*dt)-(xj+b[j].alpha1*vj*dt),ord=2)<p_new)or(b[k].x+b[k].alpha1*b[k].vx*dt>w-1-p_new) or (b[k].x+b[k].alpha1*b[k].vx*dt<p_new)or(b[k].y+b[k].alpha1*b[k].vy*dt>w-1-p_new) or (b[k].y+b[k].alpha1*b[k].vy*dt<p_new)):
        if(npl.norm((xk+b[k].alpha1*vk*dt)-(xj+b[j].alpha1*vj*dt),ord=2)<p_new):
          tcrit=(p_new-npl.norm((xk+b[k].alpha1*vk*dt)-(xj+b[j].alpha1*vj*dt),ord=2))/npl.norm(vk,ord=2)
          b[k].x_prev=b[k].x+b[k].vx*tcrit
          b[k].x=b[k].x_prev-b[k].vx*(dt-tcrit)
          b[k].y_prev=b[k].y+b[k].vy*tcrit
          b[k].y=b[k].y_prev-b[k].vy*(dt-tcrit)
          vk=vk+b[k].alpha5*(xk-xj)
          b[k].vx=vk[0]
          b[k].vy=vk[1]
        if((b[k].x+b[k].alpha1*b[k].vx*dt>w-1-p_new) or (b[k].x+b[k].alpha1*b[k].vx*dt<p_new)):
          tcrit=((w-1-p_new-b[k].x)/b[k].vx)*(b[k].x+b[k].alpha1*b[k].vx*dt>w-1-p_new)+((b[k].x-p_new)/b[k].vx)*(b[k].x+b[k].alpha1*b[k].vx*dt<p_new)
          b[k].x_prev=b[k].x+b[k].vx*tcrit
          b[k].x=b[k].x_prev-b[k].vx*(dt-tcrit)
          b[k].vx=-b[k].vx
        if((b[k].y+b[k].alpha1*b[k].vy*dt>w-1-p_new) or (b[k].y+b[k].alpha1*b[k].vy*dt<p_new)):
          tcrit=((w-1-p_new-b[k].y)/b[k].vy)*(b[k].y+b[k].alpha1*b[k].vy*dt>w-1-p_new)+((b[k].y-p_new)/b[k].vy)*(b[k].y+b[k].alpha1*b[k].vy*dt<p_new)
          b[k].y_prev=b[k].y+b[k].vy*tcrit
          b[k].y=b[k].y_prev-b[k].vy*(dt-tcrit)
          b[k].vy=-b[k].vy
        #draw.ellipse((b[k].x-r,b[k].y-r,b[k].x+r,b[k].y+r),fill=col2)
        #images.append(im)
    draw.ellipse((b[k].x-r,b[k].y-r,b[k].x+r,b[k].y+r),fill=col2)
  if(en.exist):
    ven=np.array([en.vx,en.vy])
    tar=np.array([b[target].x-en.x,b[target].y-en.y])
    en.vx=(en.alpha*(tar/npl.norm(tar,ord=2)))[0]
    en.vy=(en.alpha*(tar/npl.norm(tar,ord=2)))[1]
    ven=np.array([en.vx,en.vy])
    if(((xen+ven*dt)[0]>w-1-p_new) or ((xen+ven*dt)[0]<p_new)):
      ven[0]=-ven[0]
      en.x=-en.x
    if(((xen+ven*dt)[1]>w-1-p_new) or ((xen+ven*dt)[1]<p_new)):
      ven[1]=-ven[1]
      en.y=-en.y
    en.x_prev=en.x
    en.y_prev=en.y
    en.x=en.x+dt*en.vx
    en.y=en.y+dt*en.vy
    k=0
    eat=0
    while((k<len(b))and(not(eat))):
      if(npl.norm((en.x-b[k].x,en.y-b[k].y),ord=2)<p_new):
        if((k==0)and(len(b)>1)):
          b[1].alpha1=b[0].alpha1
          b[1].alpha2=b[0].alpha2
          b[1].alpha3=b[0].alpha3
          b[1].alpha4=b[0].alpha4
          b[1].alpha5=b[0].alpha5
        b.pop(k)
        eat=1
      else:
        k=k+1
    draw.ellipse((en.x-r,en.y-r,en.x+r,en.y+r),fill=col4)
    if(len(b)==0):
      loss=1
  images.append(im)
if(loss):
  t=0
  while(t<10):
    draw.rectangle((0,0,w-1,w-1),fill=col4,outline=col2)
    draw.text((w/2.5,w/2),"GAME OVER",col2)
    images.append(im)
    t=t+dt
images[0].save('pillow_imagedraw.gif',save_all=True,append_images=images[1:],optimize=False,duration=40,loop=0)

circle turn
point(glav=1, x_prev=310.75, y_prev=371.5, x=311.125, y=372.25, vx=1.0, vy=2.0, alpha1=0.5, alpha2=1.0, alpha3=0.6, alpha4=0.4, alpha5=0.0, catched=0)
point(glav=1, x_prev=310.75, y_prev=371.5, x=311.125, y=372.25, vx=-2.2278619577115863, vy=-0.19139252174915033, alpha1=0.5, alpha2=1.0, alpha3=0.6, alpha4=0.4, alpha5=0.0, catched=0)


circle turn
point(glav=1, x_prev=168.26335196174603, y_prev=359.97695454283934, x=167.4279037276042, y=359.9051823471834, vx=-2.2278619577115863, vy=-0.19139252174915033, alpha1=0.5, alpha2=1.0, alpha3=0.6, alpha4=0.4, alpha5=0.0, catched=0)
point(glav=1, x_prev=168.26335196174603, y_prev=359.97695454283934, x=167.4279037276042, y=359.9051823471834, vx=1.845801685260386, vy=-1.2621474314397347, alpha1=0.5, alpha2=1.0, alpha3=0.6, alpha4=0.4, alpha5=0.0, catched=0)


segm turn
point(glav=1, x_prev=296.8647469064889, y_prev=271.3970937174678, x=297.55692253846155, y=270.92378843067786, vx=1.845801685260386, vy=-1.2621474314397347, alpha1=0.5, al